# Named Entity Recognition

Задачи:

* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


# Предобработка текста

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import re
import nltk
import numpy as np
import pymorphy2

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daivanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [5]:
data = pd.read_csv('test_data.csv')
data.head(5)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [7]:
# Напишем функцию позволяющею подготовить, очистить текст
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('russian'))
stopwords = stopwords.union(set(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', 'угу']))

def data_preparation(text):
    
    "используем регулярные выражения для очистки текста"
        
    text = re.sub(r'[^а-яА-Я]', ' ', text)
    text = text.lower()
    
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    
    text_lemm = [lemmatizer.lemmatize(word) for word in text]
    
    clean_text = " ".join(text_lemm)
       
    return clean_text

data['clean_text'] = data['text'].apply(data_preparation)

## Вывод


Пропусков в данных нет, тип данных корректный.

Во время очистки теста:
* с помощью регулярных выражений оставляем только буквы, убираем все знаки препинания, если они есть. 
* По первым пяти строкам датасета видим, что только первое слово в реплике с большой буквы, а имена и название компании с маленькой,  поэтому следующим шагом можно привести весь текст к нижнему регистру.
* Очищаем текст от стоп-слов. 
* Ощищенные и предобработанные реплики сохраняем в отдельный столбец.


# Извлекаем имя менеджера

Извлечем все имена и запишем их в колонку `name` напротив реплики, где это имя употреблялось.

In [8]:
prob_thresh = 0.4
line_id = []
name = []

morph = pymorphy2.MorphAnalyzer()
for i in range(len(data['clean_text'])):
    text = data['clean_text'][i]
    
    for word in nltk.word_tokenize(text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                line_id.append(i)
                name.append(word)

In [9]:
name = pd.Series(data=name, index=line_id, name="name")
name

3       ангелина
111     ангелина
159        денис
167     ангелина
250      дмитрий
251       максим
253      дмитрий
253    анастасия
338    анастасия
341         дима
358    анастасия
438     вячеслав
444       максим
472      дмитрий
Name: name, dtype: object

In [10]:
df = data.join(name, how='left')

# Извлекаем реплики с приветствием – где менеджер поздоровался

Предположим, что способов поздароваться в нашем случае не очень много и будем искать вхождение приветственных слов в реплику. Так как приветствие обычно звучит в начале, установим, что номер реплики не должен превышать 3. Нас интересуют только реплики менеджера, поэтому сделаем срез по датасету.

In [11]:
data_manager_index = data.query('role == "manager"')
data_manager = data_manager_index.reset_index()

In [12]:
data_manager = data_manager.drop('index', axis=1)

In [13]:
greeting = ['здрав', 'добр', 'приве']
number_greeting = []   
for i in range(len(data_manager['text'])):
    for word in greeting:
        if word in data_manager['clean_text'][i] and data_manager['line_n'][i] < 3:
            number_greeting.append(i)
        else:
            pass

In [14]:
greeting_ser = pd.Series(data=['True']*len(number_greeting), index=number_greeting, name="greeting")
data_manager = data_manager.join(greeting_ser, how='left')

In [15]:
data_manager.head(2)

,dlg_id,line_n,role,text,clean_text,greeting
0,0,1,manager,Алло здравствуйте,алло здравствуйте,True
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,NaN


# Извлекаeм реплики, где менеджер попрощался

In [16]:
bye_word = ['свидан', 'добр', 'скоро']
number_bye = []   
for i in range(len(data_manager['text'])):
    for word in bye_word:
        if word in data_manager['clean_text'][i] and data_manager['line_n'][i] > 3:
            number_bye.append(i)
        else:
            pass

In [17]:
bye_ser = pd.Series(data=['True']*len(number_bye), index=number_bye, name="bye_word")
data_manager = data_manager.join(bye_ser, how='left')
data_manager.tail(2)

,dlg_id,line_n,role,text,clean_text,greeting,bye_word
199,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,NaN,NaN
200,5,142,manager,Ну до свидания хорошего вечера,свидания хорошего вечера,NaN,True


# Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [18]:
data_manager_group = data_manager.groupby('dlg_id')[['greeting', 'bye_word']].count()

In [19]:
data_manager_group

,greeting,bye_word
dlg_id,,
0,1,1
1,1,1
2,1,0
3,2,1
4,0,1
5,0,1


In [20]:
for i in range(1, 6):
    if data_manager_group['greeting'][i] != 0 and data_manager_group['bye_word'][i] !=0:
        print(f'В диалоге {i} менеджер поздоровался и попрощался с клиентом')
    elif (data_manager_group['greeting'][i] == 0 and data_manager_group['bye_word'][i] !=0 or
    data_manager_group['greeting'][i] != 0 and data_manager_group['bye_word'][i] ==0):
        print(f'В диалоге {i} менеджер только поздаровался или порощался')
    else:
        print('Менеджер не поздаровался и не попращался')

В диалоге 1 менеджер поздоровался и попрощался с клиентом
В диалоге 2 менеджер только поздаровался или порощался
В диалоге 3 менеджер поздоровался и попрощался с клиентом
В диалоге 4 менеджер только поздаровался или порощался
В диалоге 5 менеджер только поздаровался или порощался


## Вывод:

* Из 6 диалогов только в 3-х менеджер поздоровался и попрощался.

# Извлекаeм название компании

Предположим, что когда менеджер или клиент говорит название компании он употребляет слово компания. Будем искать реплики по вхождению этого слова.

In [21]:
company = ['компани', 'организац', 'фирма']
number_company = []
company_name=[]
for i in range(len(df['text'])-1):
    for word in company:
        if word in df['clean_text'][i]:
            number_company.append(i)
            
            t = df['clean_text'][i]
            name = ' '.join(t.split(word)[1].split(' ')[1:3])# 3 words 
            company_name.append(name)
        else:
            pass


In [22]:
company_name

['диджитал бизнес',
 'которая нам',
 'которые звонят',
 'которой работаю',
 'поэтому',
 'даю просто',
 'которая принципе',
 '',
 'которые',
 '',
 '',
 'которая работает',
 'которая звонит',
 '',
 'диджитал бизнес',
 'диджитал бизнес',
 'называется',
 'китобизнес удобно',
 'которые внедрить',
 '',
 'интеграции сделаны']

In [23]:
com_ser = pd.Series(data= company_name[0:4], index=number_company[0:4], name="company")
df = df.join(com_ser, how='left')
df.head(5)

,dlg_id,line_n,role,text,clean_text,name,company
0,0,0,client,Алло,алло,NaN,NaN
1,0,1,manager,Алло здравствуйте,алло здравствуйте,NaN,NaN
2,0,2,client,Добрый день,добрый день,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,ангелина,диджитал бизнес
4,0,4,client,Ага,ага,NaN,NaN


## Вывод

* Наш способ работает далеко неидельно, человек может употреблять слово компания не называя саму компанию или давать общую характеристику компании, например, транспортная компания.

# Извлекаем реплики, где менеджер представил себя.

Вероятнее всего менеджер представил себя в репликах, где есть имена.

In [24]:
introduce_word = ['зову', 'это ', ' я ']
number_introduce = []   
for i in range(len(data_manager['text'])):
    for word in introduce_word:
        if word in data_manager['clean_text'][i]:
            number_introduce.append(i)
        else:
            pass

In [25]:
introduce_ser = pd.Series(data=['True']*len(number_introduce), index=number_introduce, name="introduce_yourself")
data_manager = data_manager.join(introduce_ser, how='left')
data_manager.head(5)

,dlg_id,line_n,role,text,clean_text,greeting,bye_word,introduce_yourself
0,0,1,manager,Алло здравствуйте,алло здравствуйте,True,NaN,NaN
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,NaN,NaN,True
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,возможно рассмотрите другие варианты видите хо...,NaN,NaN,NaN
3,0,8,manager,Угу а на что вы обращаете внимание при выборе,обращаете внимание выборе,NaN,NaN,NaN
4,0,11,manager,Что для вас приоритет,приоритет,NaN,NaN,NaN


# Объединяем полученную информацию

Соберем всю полученную информацию в итоговый датасет.

In [27]:
index = list(data_manager_index.index)

In [51]:
data_manager_f = data_manager[['greeting', 'bye_word', 'introduce_yourself']]
data_manager_f['index'] = index
data_manager_f = data_manager_f.set_index('index')

In [52]:
data_final = df.join(data_manager_f, how='left')

In [55]:
data_final[['greeting',
              'bye_word',
              'introduce_yourself']] = np.where(data_final[['greeting',
                                                             'bye_word',
                                                             'introduce_yourself']] == "True", True, False)

In [57]:
data_final.head(10)

,dlg_id,line_n,role,text,clean_text,name,company,greeting,bye_word,introduce_yourself
0,0,0,client,Алло,алло,NaN,NaN,False,False,False
1,0,1,manager,Алло здравствуйте,алло здравствуйте,NaN,NaN,True,False,False
2,0,2,client,Добрый день,добрый день,NaN,NaN,False,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,ангелина,диджитал бизнес,False,False,True
4,0,4,client,Ага,ага,NaN,NaN,False,False,False
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,возможно рассмотрите другие варианты видите хо...,NaN,NaN,False,False,False
6,0,6,client,Да мы работаем с компанией которая нам подлива...,работаем компанией которая нам подливает поэто...,NaN,которая нам,False,False,False
7,0,7,client,Как как бы уже до этого момента работаем все у...,момента работаем устраивает сопровождение поэтому,NaN,NaN,False,False,False
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,обращаете внимание выборе,NaN,NaN,False,False,False
9,0,9,client,Как бы нет,,NaN,NaN,False,False,False


# Возникшие сложности

* **При извлечении имен и названий компаний**

* При использовании библиотеки `spaCy` можно адекватно извлечь имена в репликах, но название компании таким способом извлечь не получилось. Основная сложность в том, что название компании написано с маленькой буквы и никак не обозначено пунктуацией. Поэтому отличить название от другого любого текста в этих репликах на данном этапе работы над проектом, получилось только способом описанным выше.
* Менеджер может называть не только свое имя, но и имя клиента или коллеги. В данном случае, очень сложно понять какое конкретно имя соответствует имени менееджера. В одном из реплик датасета, менеджер упоминает имя клиента, имя своей колллеги, но не называет своего имени. Так же и клиент может называть как свое имя, когда представляется, так и имя менеджера, когда к нему обращается.

In [37]:
import spacy
from spacy.lang.ru import Russian

In [38]:
nlp = spacy.load("ru_core_news_md")
for i in range(len(data['clean_text'])):
    doc = nlp(data['clean_text'][i])
    for ent in doc.ents:
         print(ent.text, ent.label_)

ангелина PER
хирам PER
э PER
делайте аудиты PER
ангелина PER
ватсап PER
денис дэ е эн PER
ангелина PER
кэшбэка PER
ватсап почту PER
сапаров PER
астана LOC
кз ORG
жесупов PER
дмитрий PER
максим PER
дмитрий коллега анастасия PER
сэйл ботом PER
инбокс PER
амо ORG
амо ORG
си эр эм PER
айдар PER
главное LOC
дима PER
ла ла ла PER
анастасия PER
амо ORG
хэлп PER
вячеслав PER
максим PER
дмитрий PER


При использовании библиотеки `natasha` извлеть какие-липбо сущности на данном этапе выполнения задания не удалось.

In [58]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

for i in range(len(data['text'])):
    text = data['text'][i]
    doc = Doc(text)
    
print(doc.spans)

None


#  Возможное улучшение решения задачи

* Более детально разобраться в настройках библиотека `spaCy`, возможно этот позволит улучшить результат извлечения сущностей.
* Использовать предобученную нейроную сеть (BERT).
* Изучить как `pymorphy2` может помочь в извлечении реплик с приветствием и прощанием.

# Вывод

В результате выполнения задания получили новый датасет, в котором в колонке 
* `name` - имя, упомянутое в ремплике,
* `company` - название компании, упомянутое в реплике,
* `greeting`- есть ли в реплике приветствие (если да - True, если нет - False),
* `bye_word`- если ли в реплике слова прощания (если да - True, если нет - False),
* `introduce_yourself` - представил ли себя менеджер в этой реплике (если да - True, если нет - False).